# Continue Pretraining on T5

In [1]:
from datasets import load_from_disk

tokenized_dataset = load_from_disk('/home/j/Desktop/MLotsawa/Notebooks/Models/TibetanToEnglishTranslation/TibetanToEnglishTranslationv2/Ground/BuddhistOnly/Pretraining/Data/tokenized-pretraining-ds')

Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('billingsmoore/getok-v0')

## Train the Model

In [3]:
%env WANDB_PROJECT=english-v2

env: WANDB_PROJECT=english-v2


In [4]:
from transformers import TrainingArguments, Trainer, T5ForConditionalGeneration, EarlyStoppingCallback

model = T5ForConditionalGeneration.from_pretrained("google/t5-efficient-tiny", device_map='cuda:0')
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir="efficient-tiny-pretrain",
    auto_find_batch_size=True,
    learning_rate=3e-4,
    num_train_epochs=1
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

In [5]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.699400
1000,0.374200
1500,0.349000
2000,0.320200
2500,0.308500
3000,0.304500
3500,0.296500
4000,0.287300
4500,0.275300
5000,0.266600


TrainOutput(global_step=215355, training_loss=0.1947256339213273, metrics={'train_runtime': 14378.0451, 'train_samples_per_second': 119.824, 'train_steps_per_second': 14.978, 'total_flos': 1.9438632486567936e+16, 'train_loss': 0.1947256339213273, 'epoch': 1.0})

In [6]:
model.save_pretrained('Models/pretrained-tiny-model')